In [14]:
import torch

In [15]:
from StyleTransferModel_128 import StyleTransferModel

In [16]:
import onnx
from onnx import numpy_helper
onnx_model   = onnx.load("models/inswapper_128_batched.onnx")
INTIALIZERS  = onnx_model.graph.initializer
onnx_weights = {}
for initializer in INTIALIZERS:
    W = numpy_helper.to_array(initializer)
    onnx_weights[initializer.name] = W

In [17]:
model = StyleTransferModel()

In [18]:
weight_shapes = []
for n, p in model.named_parameters():
    weight_shapes.append((n, '-'.join([str(x) for x in list(p.shape)])))

In [19]:
for k, v in model.state_dict().items():
    print(k)
    print(v.shape, '\n')

down.0.weight
torch.Size([128, 3, 7, 7]) 

down.0.bias
torch.Size([128]) 

down.2.weight
torch.Size([256, 128, 3, 3]) 

down.2.bias
torch.Size([256]) 

down.4.weight
torch.Size([512, 256, 3, 3]) 

down.4.bias
torch.Size([512]) 

down.6.weight
torch.Size([1024, 512, 3, 3]) 

down.6.bias
torch.Size([1024]) 

style_blocks.0.conv1.conv.weight
torch.Size([1024, 1024, 3, 3]) 

style_blocks.0.conv1.conv.bias
torch.Size([1024]) 

style_blocks.0.conv1.adain.style.weight
torch.Size([2048, 512]) 

style_blocks.0.conv1.adain.style.bias
torch.Size([2048]) 

style_blocks.0.conv2.conv.weight
torch.Size([1024, 1024, 3, 3]) 

style_blocks.0.conv2.conv.bias
torch.Size([1024]) 

style_blocks.0.conv2.adain.style.weight
torch.Size([2048, 512]) 

style_blocks.0.conv2.adain.style.bias
torch.Size([2048]) 

style_blocks.1.conv1.conv.weight
torch.Size([1024, 1024, 3, 3]) 

style_blocks.1.conv1.conv.bias
torch.Size([1024]) 

style_blocks.1.conv1.adain.style.weight
torch.Size([2048, 512]) 

style_blocks.1.conv1.a

In [20]:
for k, v in onnx_weights.items():
    print(k)
    print(v.shape, '\n')

styles.0.conv1.1.weight
(1024, 1024, 3, 3) 

styles.0.conv1.1.bias
(1024,) 

styles.0.style1.linear.weight
(2048, 512) 

styles.0.style1.linear.bias
(2048,) 

styles.0.conv2.1.weight
(1024, 1024, 3, 3) 

styles.0.conv2.1.bias
(1024,) 

styles.0.style2.linear.weight
(2048, 512) 

styles.0.style2.linear.bias
(2048,) 

styles.1.conv1.1.weight
(1024, 1024, 3, 3) 

styles.1.conv1.1.bias
(1024,) 

styles.1.style1.linear.weight
(2048, 512) 

styles.1.style1.linear.bias
(2048,) 

styles.1.conv2.1.weight
(1024, 1024, 3, 3) 

styles.1.conv2.1.bias
(1024,) 

styles.1.style2.linear.weight
(2048, 512) 

styles.1.style2.linear.bias
(2048,) 

styles.2.conv1.1.weight
(1024, 1024, 3, 3) 

styles.2.conv1.1.bias
(1024,) 

styles.2.style1.linear.weight
(2048, 512) 

styles.2.style1.linear.bias
(2048,) 

styles.2.conv2.1.weight
(1024, 1024, 3, 3) 

styles.2.conv2.1.bias
(1024,) 

styles.2.style2.linear.weight
(2048, 512) 

styles.2.style2.linear.bias
(2048,) 

styles.3.conv1.1.weight
(1024, 1024, 3, 3) 

s

In [21]:
replacement_dict = {
    'styles': 'style_blocks',
    'conv1.1': 'conv1.conv',
    'style1.linear': 'conv1.adain.style',
    'conv2.1': 'conv2.conv',
    'style2.linear': 'conv2.adain.style',
    'up0.1': 'up.8',
    'onnx::Conv_833': 'down.0.weight',
    'onnx::Conv_834': 'down.0.bias',
    'onnx::Conv_836': 'down.2.weight',
    'onnx::Conv_837': 'down.2.bias',
    'onnx::Conv_839': 'down.4.weight',
    'onnx::Conv_840': 'down.4.bias',
    'onnx::Conv_842': 'down.6.weight',
    'onnx::Conv_843': 'down.6.bias',
    'onnx::Conv_845': 'up.1.weight',
    'onnx::Conv_846': 'up.1.bias',
    'onnx::Conv_848': 'up.4.weight',
    'onnx::Conv_849': 'up.4.bias',
    'onnx::Conv_851': 'up.6.weight',
    'onnx::Conv_852': 'up.6.bias',
    # 'initializer': 'initializer.weight'
}

In [22]:
renamed_weights = {}

for k, v in onnx_weights.items():
    orig_k = k
    for name, replacement in replacement_dict.items():
        k = k.replace(name, replacement)

    if k == orig_k:
        shape_name = '-'.join([str(x) for x in v.shape])
        print(f'Shape of {k}: {v.shape}')
        print(f'Possible names:')
        replacements = []
        for weight_shape in weight_shapes:
            if shape_name == weight_shape[-1]:
                print(f'  {weight_shape[0]}')
                replacements.append(weight_shape[0])
        if len(replacements) == 1:
            k = replacements[0]

    if k != orig_k:
        renamed_weights[k] = v

Shape of onnx::Resize_918: (4,)
Possible names:
Shape of onnx::Pad_122: (8,)
Possible names:
Shape of onnx::Pad_157: (8,)
Possible names:
Shape of onnx::Add_164: ()
Possible names:
Shape of onnx::Div_167: ()
Possible names:
Shape of onnx::Gather_174: (1,)
Possible names:
Shape of onnx::Slice_176: (1,)
Possible names:
Shape of onnx::Resize_787: (0,)
Possible names:
Shape of onnx::Div_830: ()
Possible names:
Shape of onnx::Slice_182: (1,)
Possible names:
Shape of onnx::Slice_185: (1,)
Possible names:
Shape of buff2fs: (512, 512)
Possible names:
Shape of initializer: (512, 512)
Possible names:


In [23]:
state_dict = {k: torch.from_numpy(v) for k, v in renamed_weights.items()}

model.load_state_dict(state_dict)

<All keys matched successfully>

In [24]:
torch.save(model.state_dict(), 'inswapper_128_dict.pt')

In [25]:
torch.save(model, 'inswapper_128_cleaned.pt')

In [26]:
model(torch.rand((1, 3, 128, 128)), torch.rand((1, 512))).shape

torch.Size([1, 3, 256, 256])

In [29]:
tgt = torch.randn(1, 3, 128, 128)
src_e = torch.randn(1, 512)

y = model(tgt, src_e)

model.eval()

torch.onnx.export(
    model, 
    (tgt, src_e), 
    "inswapper_eg.onnx", 
    export_params=True,
    opset_version=11, 
    input_names=['target', 'source'], 
    output_names=['output'], 
    dynamic_axes={'target': {0: 'batch_size'}, 'source': {0: 'batch_size'}, 'output': {0: 'batch_size'}}
)